### Training Process

### Import Necessary Libraries

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D, Flatten, MaxPooling2D
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [ ]:
### Importing Dataset

In [6]:
dataset=pd.read_csv('dataset/fer20131.csv')
dataset

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest


#### Selecting Particluar emotions to train 

In [3]:
selected_emotions = [0, 2, 3, 4, 5]  # Emotions to include

# Assign the Labels

emotion_labels = ['Angry', 'Fear', 'Happy', 'Sad', 'Surprise']

Empyty list to retrieve each image and label in array format

In [8]:
images = []
labels = []

#### Retrieving Images and Pre-processing

In [9]:
for i in range(len(dataset)):
    if dataset['emotion'][i] in selected_emotions:
        pixels = np.array(dataset['pixels'][i].split(' '), dtype='float32')
        image = pixels.reshape((48, 48, 1)) / 255.0
        images.append(image)
        labels.append(dataset['emotion'][i])

In [10]:
images = np.array(images)
labels = np.array(labels)

### Map emotion labels to corresponding indices 

In [11]:
label_mapping = {emotion: idx for idx, emotion in enumerate(selected_emotions)}
labels_mapped = np.array([label_mapping[label] for label in labels])

# Convert labels to categorical
num_classes = len(selected_emotions)
labels_categorical = to_categorical(labels_mapped, num_classes=num_classes)

### Split Dataset for training & testing

In [12]:
feature_train, feature_test, label_train, label_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=7)

### Create model

In [13]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(5, 5), input_shape=(48, 48, 1), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(5, 5), activation='relu'))
model.add(Conv2D(128, kernel_size=(5, 5), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, kernel_size=(5, 5), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

### Compile the model

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

### Train model

In [17]:
model.fit(feature_train, label_train, batch_size=64,epochs=15, verbose=1, validation_data=(feature_test, label_test), shuffle=True)

Epoch 1/15
365/365 [==============================] - 281s 771ms/step - loss: 1.3883 - accuracy: 0.4156 - val_loss: 1.7005 - val_accuracy: 0.3037
Epoch 2/15
365/365 [==============================] - 279s 766ms/step - loss: 1.1550 - accuracy: 0.5276 - val_loss: 1.3568 - val_accuracy: 0.4237
Epoch 3/15
365/365 [==============================] - 301s 824ms/step - loss: 1.0044 - accuracy: 0.5943 - val_loss: 1.2077 - val_accuracy: 0.5222
Epoch 4/15
365/365 [==============================] - 313s 858ms/step - loss: 0.8869 - accuracy: 0.6487 - val_loss: 1.0622 - val_accuracy: 0.5745
Epoch 5/15
365/365 [==============================] - 312s 855ms/step - loss: 0.7638 - accuracy: 0.6960 - val_loss: 1.1390 - val_accuracy: 0.5660
Epoch 6/15
365/365 [==============================] - 312s 856ms/step - loss: 0.6291 - accuracy: 0.7565 - val_loss: 1.2060 - val_accuracy: 0.5716
Epoch 7/15
365/365 [==============================] - 288s 789ms/step - loss: 0.4882 - accuracy: 0.8153 - val_loss: 1.1953 -

### Save Model

These three methods are Preferred choices

In [19]:
model.save('face_cnn_model.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
model.save_weights("face_cnn_model_weights.h5")

In [21]:
model_json = model.to_json()
with open('face_cnn_model_architecture.json', 'w') as json_file:
    json_file.write(model_json)

### Model is Trained and saved , if you face any warnings in the notebook ignore them